In [25]:
import numpy as np
import pandas as pd
import rao_utilities as util
import rao_plotting as plot
import rao_models as model
import rao_math as m
import warnings
import importlib

from numpy import sqrt
from feets import ExtractorWarning
from pathlib import PurePath

importlib.reload(util)
importlib.reload(plot)
importlib.reload(model)
importlib.reload(m)

## CONSTANTS ============================================================================
# FILENAME = 'data/data.csv'
FILENAME = 'data/M3_raw01_Photometry25-47-59.csv' 
# FILENAME = 'data/M3_raw02_Photometry25-47-59.csv'
# FILENAME = 'data/M3_raw03_Photometry28-47-55.csv'
# FILENAME = 'data/M3_raw04_Photometry35-47-55.csv'
# FILENAME = 'data/M3_raw01_Photometry28_47_55_may_29.csv' # Change me to work with different files/filename
# FILENAME = 'data/M3_2nights_rawPhotometry.csv'
# FILENAME = 'data/M3_night_1.xlsx'
# FILENAME = 'data/M3_night_2.xlsx'

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) # Need this to prevent it from spamming
warnings.filterwarnings("ignore", category=ExtractorWarning)
## FUNCTIONS ============================================================================

# I moved everything into different python modules since this was getting extremely large

## CALCULATION ============================================================================
file = PurePath(FILENAME)

df = util.extract_data(FILENAME)
# bad_rows = df[df['name'] == 'M3-12']
# df = df.drop(index=bad_rows.index)

mags = df['mag'].to_numpy(dtype='float64')
error = df['error'].to_numpy(dtype='float64')

num_stars, samples = util.extract_samples_stars(df)
if 'jd' in df.columns:
    timeline = df['jd'].unique()
    df['jd'] = pd.to_datetime(df['jd'], origin='julian', unit='D')
else: 
    timeline = np.linspace(0, samples*EXPOSURE_TIME, samples)

mags = np.reshape(mags, (samples, num_stars))      # Stack both mags and error according to time, new column = new file in time
error = np.reshape(error, (samples, num_stars))
diff_mags = m.subtract_all_magnitudes(mags)          # array of arrays, each array corresponding to a single star as the 'target'
uncertainties = m.calculate_all_uncertainties(error) # Each individual array is organized by incrementing time in columns, rows are stars
average_diff_mags = np.mean(diff_mags, axis=1, dtype=np.float64) # mean of every column, accumulating all stars at particular time
average_uncertainties = sqrt(np.sum(uncertainties**2, axis=1, dtype=np.float64))/uncertainties[0].shape[0] # sum of squares of each column, then sqrt'd

df['average_diff_mags'] = average_diff_mags.T.reshape(samples*num_stars, 1)
df['average_uncertainties'] = average_uncertainties.T.reshape(samples*num_stars, 1)

box_model_results = model.box_least_squares(average_diff_mags, average_uncertainties, timeline=timeline)

# # Feature Extraction information
# indices = m.calculate_all_feets_indices(data = average_diff_mags, 
#                                           timeline = timeline, 
#                                           uncertainties= average_uncertainties)
# feature_df = pd.DataFrame(indices)
# feature_df.to_excel('Features_of_stars.xlsx')

# # Data export for Dr. Langill
# indices_of_stars = [ sub['name'] for sub in box_model_results]
# original_data_filtered = df.iloc[indices_of_stars]
# original_data_filtered.to_excel('Stars_with_varying_light.xlsx')

# # Chi squared test
chi = model.chi_squared(mags.T, error.T, timeline)
print(chi)
# pd.DataFrame({'chisquared':chi}).to_excel('chi_squared_test.xlsx')

# plot.plot_and_save_all(df, file.stem)

[ 82.5688118   42.83218453   6.67514935  11.0463576  143.09091118
  59.39723694  11.78681331   4.18879671   4.33205281   2.30495986
  20.07603213  52.07801702   4.15563301   6.10396308   6.50044851
  10.10548311   3.52918289   4.71359178   3.06957775   3.65382969
   4.10970365   4.29947194   2.81150419   6.13499016   4.85813542
   5.11108465   9.14754834   5.98030997   5.35877155   5.3429034
   4.66174233   4.6546779    4.06055268   2.72439357   1.88941989
  10.90348573   3.49488672   5.09545972   5.07589592   5.5652656
  14.68745642   4.56335926   6.66787971  16.64792891   5.43309406
   5.67794736   5.78337234   3.83761719   6.17848631   3.24599673
   8.34132506   3.78396004   3.37731693   4.67624591   4.38079765
   9.74758284   6.17907656   4.4712575    4.9946243    6.0526294
   3.0963203   11.15728684   3.81471611   5.79906946  10.73128945
   4.48792266   4.77781768   3.54170101   5.24180088  11.03359977
   3.05211699   3.31410121   6.08923776   4.05470842   4.81328449
   8.32955975